<a href="https://colab.research.google.com/github/LeeJaeEun0/PlayData_230118/blob/main/230118_ch06_DL_04_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM
* 주가, 날씨, 텍스트 등 순서가 있는 데이터에 사용하기 좋음
* RNN보다 성능이 좋음

In [1]:
# 뉴욕 타임스 코멘트 데이터
# https://www.kaggle.com/datasets/aashita/nyt-comments
# https://www.kaggle.com/{본인ID}/account
# 본인 프로필 누르고 > Account
# > Create New API Token > kaggle.json
from google.colab import files
!pip install -q kaggle #kaggle 설치 # --quiet
# -- kaggle api를 사용하기 위한 인증파일을 설정 (8-11)
files.upload() #kaggle API file upload
!mkdir ~/.kaggle # kaggle 디렉토리 생성 / mkdir : make directory - 폴더 생성 (~)
!cp kaggle.json ~/.kaggle/ #kaggle.json 파일 kaggle 폴더에 복사 / cp a b (copy)
!chmod 600 ~/.kaggle/kaggle.json # 권한 변경 r w x (4 2 1)
# ---------
# !kaggle datasets list #kaggle 데이터셋 리스트 체크
# https://www.kaggle.com/datasets/aashita/nyt-comments
# ... > Copy API command > 예) kaggle datasets download -d aashita/nyt-comments
!kaggle datasets download -d aashita/nyt-comments
!ls # 다운받은 파일 리스트 확인 -> nyt-comments.zip

Saving kaggle.json to kaggle.json
100% 479M/480M [00:21<00:00, 24.5MB/s]
100% 480M/480M [00:21<00:00, 23.3MB/s]
kaggle.json  nyt-comments.zip  sample_data


In [2]:
!unzip nyt-comments

Archive:  nyt-comments.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


In [3]:
import pandas as pd

df = pd.read_csv('ArticlesApril2017.csv')

In [4]:
df.columns # 사람이 작성한 자연어 데이터 'headline'만 사용

Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [5]:
df.headline

0      Finding an Expansive View  of a Forgotten Peop...
1                      And Now,  the Dreaded Trump Curse
2                  Venezuela’s Descent Into Dictatorship
3                  Stain Permeates Basketball Blue Blood
4                              Taking Things for Granted
                             ...                        
881                  Reporting on Gays Who ‘Don’t Exist’
882    The Fights That Could Lead to a Government Shu...
883    ‘The Leftovers’ Season 3, Episode 2: Swedish P...
884                          Thinking Out Loud, But Why?
885                    Some Sugar. Could Use More Spice.
Name: headline, Length: 886, dtype: object

# 데이터 전처리

1. 특수문자 제거
2. BOW(Bag of words - 모든 단어를 겹치지않도록 고유한 번호로 나타낸 집합 = 단어 사전) 

In [6]:
import numpy as np
import glob # 파일을 패턴화해서 읽을 수 있게 도와주는 라이브러리

from torch.utils.data.dataset import Dataset
import string # 특수문제 제거용 (punctuation)

In [7]:
# 특수문자 제거용 함수 정의
# -> 모든 문자를 소문자로 바꾸고 특수문자를 제거
def clean_text(txt):
    new_text = []
    for v in txt: # 문자 한 개씩 반환
        if v not in string.punctuation: # 특수문자가 아닌 것만 남기기
            new_text.append(v)
    return "".join(new_text).lower() # 소문자화

In [8]:
clean_text('Hello. Hello> Hello!!!')

'hello hello hello'

In [9]:
glob.glob('*.csv') # 특정 확장자를 가진 모든 파일 검색

['ArticlesFeb2017.csv',
 'ArticlesMarch2018.csv',
 'ArticlesJan2017.csv',
 'CommentsApril2017.csv',
 'ArticlesFeb2018.csv',
 'ArticlesJan2018.csv',
 'ArticlesMay2017.csv',
 'CommentsMarch2017.csv',
 'ArticlesApril2017.csv',
 'CommentsFeb2017.csv',
 'ArticlesApril2018.csv',
 'CommentsJan2018.csv',
 'CommentsJan2017.csv',
 'CommentsApril2018.csv',
 'ArticlesMarch2017.csv',
 'CommentsMay2017.csv',
 'CommentsFeb2018.csv',
 'CommentsMarch2018.csv']

In [10]:
glob.glob('Articles*.csv') # 특정한 확장자와 접두사를 가진 모든 파일 검색

['ArticlesFeb2017.csv',
 'ArticlesMarch2018.csv',
 'ArticlesJan2017.csv',
 'ArticlesFeb2018.csv',
 'ArticlesJan2018.csv',
 'ArticlesMay2017.csv',
 'ArticlesApril2017.csv',
 'ArticlesApril2018.csv',
 'ArticlesMarch2017.csv']

In [11]:
glob.glob('*2017.csv')

['ArticlesFeb2017.csv',
 'ArticlesJan2017.csv',
 'CommentsApril2017.csv',
 'ArticlesMay2017.csv',
 'CommentsMarch2017.csv',
 'ArticlesApril2017.csv',
 'CommentsFeb2017.csv',
 'CommentsJan2017.csv',
 'ArticlesMarch2017.csv',
 'CommentsMay2017.csv']

In [12]:
glob.glob('Articles*2017.csv')

['ArticlesFeb2017.csv',
 'ArticlesJan2017.csv',
 'ArticlesMay2017.csv',
 'ArticlesApril2017.csv',
 'ArticlesMarch2017.csv']

In [13]:
glob.glob('*May.csv')

[]

In [14]:
glob.glob('*May*.csv')

['ArticlesMay2017.csv', 'CommentsMay2017.csv']

# 데이터셋

In [15]:
all_headlines = []
# Articles로 시작하는 csv파일 이름을 모두 검색해서 가져오는 코드
for filename in glob.glob("Articles*.csv"):
    article_df = pd.read_csv(filename) # 파일이름으로 csv를 df로 만들어줌
    # df.headline : headline이라는 이름의 컬럼(열)만 불러오기
    # -> index, value 시리즈 -> values -> 인덱스없이 '값'만 불러온 ndarray
    # -> list (  ) -> 리스트 타입으로 바뀜 -> all_headlines와 연결
    all_headlines.extend(list(article_df.headline.values))
    # all_headlines += list(article_df.headline.values)

In [16]:
all_headlines

['N.F.L. vs. Politics Has Been Battle All Season Long',
 'Voice. Vice. Veracity.',
 'A Stand-Up’s Downward Slide',
 'New York Today: A Groundhog Has Her Day',
 'A Swimmer’s Communion With the Ocean',
 'Trail Activity',
 'Super Bowl',
 'Trump’s Mexican Shakedown',
 'Pence’s Presidential Pet',
 'Fruit of a Poison Tree',
 'The Peculiar Populism of Donald Trump',
 'Questions for: ‘On Alaska’s Coldest Days, a Village Draws Close for Warmth’',
 'The New Kids',
 'What My Chinese Mother Made',
 'Do You Think Teenagers Can Make a Difference in the World?',
 'Unknown',
 'President Pledges to Let Politics Return to Pulpits',
 'The Police Killed My Unarmed Son in 2012. I’m Still Waiting for Justice.',
 'Video of Sheep Slaughtering Ignites a Dispute',
 'This Will Change Your Mind',
 'Busy Start for a President, and That Was in 1933',
 'Trump Reverts to Pillars of Obama Policies Abroad',
 'Should Dollar Rise or Fall? The Trump Team’s Message Is Garbled',
 'Hardware for the Modern Winemaker',
 'Cajun

In [17]:
all_headlines = [ h for h in all_headlines if h != 'Unknown']

In [18]:
all_headlines

['N.F.L. vs. Politics Has Been Battle All Season Long',
 'Voice. Vice. Veracity.',
 'A Stand-Up’s Downward Slide',
 'New York Today: A Groundhog Has Her Day',
 'A Swimmer’s Communion With the Ocean',
 'Trail Activity',
 'Super Bowl',
 'Trump’s Mexican Shakedown',
 'Pence’s Presidential Pet',
 'Fruit of a Poison Tree',
 'The Peculiar Populism of Donald Trump',
 'Questions for: ‘On Alaska’s Coldest Days, a Village Draws Close for Warmth’',
 'The New Kids',
 'What My Chinese Mother Made',
 'Do You Think Teenagers Can Make a Difference in the World?',
 'President Pledges to Let Politics Return to Pulpits',
 'The Police Killed My Unarmed Son in 2012. I’m Still Waiting for Justice.',
 'Video of Sheep Slaughtering Ignites a Dispute',
 'This Will Change Your Mind',
 'Busy Start for a President, and That Was in 1933',
 'Trump Reverts to Pillars of Obama Policies Abroad',
 'Should Dollar Rise or Fall? The Trump Team’s Message Is Garbled',
 'Hardware for the Modern Winemaker',
 'Cajun, Far From H

In [19]:
class TextGeneration(Dataset):
    def clean_text(self, txt):
        # new_text = []
        # for v in txt: # 문자 한 개씩 반환
        #     if v not in string.punctuation: # 특수문자가 아닌 것만 남기기
        #         new_text.append(v)
        # return "".join(new_text).lower() # 소문자화
        return "".join([v for v in txt if v not in string.punctuation]).lower()
    # 생성자
    def __init__(self) -> None:
        super().__init__()
        # headline 사용 (나머지는 메타데이터라 의미 X)
        # Article만 사용
        all_headlines = []
        # Articles로 시작하는 csv파일 이름을 모두 검색해서 가져오는 코드
        for filename in glob.glob("Articles*.csv"):
            article_df = pd.read_csv(filename) # 파일이름으로 csv를 df로 만들어줌
            # df.headline : headline이라는 이름의 컬럼(열)만 불러오기
            # -> index, value 시리즈 -> values -> 인덱스없이 '값'만 불러온 ndarray
            # -> list (  ) -> 리스트 타입으로 바뀜 -> all_headlines와 연결
            all_headlines.extend(list(article_df.headline.values))
            # all_headlines += list(article_df.headline.values)
        # headline 중 unknown을 제거 (필터링)
        all_headlines = [h for h in all_headlines if h != 'Unknown']
        # 특수문자 등 텍스트 전처리
        self.corpus = [self.clean_text(x) for x in all_headlines]
        # corpus : 말뭉치
        # BOW : Bag of Words => 등장하는 모든 단어에 고유번호 지정
        self.BOW = {}
        for line in self.corpus: # corpus에서 headline에서 추출한 문장들을 불러오기
            for word in line.split(): # 띄어쓰기를 기준으로 토큰화 -> 단어
                if word not in self.BOW.keys(): # 단어가 BOW에 있니?
                    self.BOW[word] = len(self.BOW.keys())
        # 모델의 입력으로 사용할 데이터(self.data)
        self.data = self.generate_sequence(self.corpus)
        # sequence -> (1번, 2번) => (3번)

    # 텍스트 시계열 -> BOW
    # (단어1, 단어2) => (단어3)
    def generate_sequence(self, txt):
        seq = [] #([a, b], c)
        for line in txt: # 전처리가 된 self.corpus 데이터 = 헤드라인
            # 1단계
            line = line.split() # 토큰화
            # BOW(사전) -> key -> 고유번호 지정
            # apple -> 1, is -> 2, delicious -> 3
            line_bow = [self.BOW[word] for word in line] # 1, 2, 3 -> 문장을 고유번호화
            # 단어 2개를 입력 -> 그 다음 단어가 정답(예측 대상)
            # data = []
            # for i in range(len(line_bow)-2):
            #     # 0 -> 0, 1 => 2 / 1 -> 1, 2 => 3 / ... / n-3 -> n-2, n-1 => n-1
            #     # d = ([a, b], c)
            #     d = ([line_bow[i], line_bow[i+1]], line_bow[i+2]) # a, b를 사용해서, c를 예측 (순서대로)
            #     data.append(d)
            data = [([line_bow[i], line_bow[i+1]], line_bow[i+2])
                    for i in range(len(line_bow)-2)]
            # seq.extend(data)
            seq += data
        return seq
    
    # 데이터 개수를 반환하는 함수
    def __len__(self):
        return len(self.data)
    
    # 데이터를 불러오는 함수
    def __getitem__(self, i):
        data = np.array(self.data[i][0]) # (1번, 2번 단어)
        label = np.array(self.data[i][1]) # (3번 단어)
        return data, label

In [20]:
# class TextGeneration(Dataset):
#     def clean_text(self, txt):
#         # new_text = []
#         # for v in txt: # 문자 한 개씩 반환
#         #     if v not in string.punctuation: # 특수문자가 아닌 것만 남기기
#         #         new_text.append(v)
#         # return "".join(new_text).lower() # 소문자화
#         return "".join([v for v in txt if v not in string.punctuation]).lower()
#     # 생성자
#     def __init__(self) -> None:
#         super().__init__()
#         # headline 사용 (나머지는 메타데이터라 의미 X)
#         # Article만 사용
#         all_headlines = []
#         # Articles로 시작하는 csv파일 이름을 모두 검색해서 가져오는 코드
#         for filename in glob.glob("Articles*.csv"):
#             article_df = pd.read_csv(filename) # 파일이름으로 csv를 df로 만들어줌
#             # df.headline : headline이라는 이름의 컬럼(열)만 불러오기
#             # -> index, value 시리즈 -> values -> 인덱스없이 '값'만 불러온 ndarray
#             # -> list (  ) -> 리스트 타입으로 바뀜 -> all_headlines와 연결
#             all_headlines.extend(list(article_df.headline.values))
#             # all_headlines += list(article_df.headline.values)
#         # headline 중 unknown을 제거 (필터링)
#         all_headlines = [h for h in all_headlines if h != 'Unknown']
#         # 특수문자 등 텍스트 전처리
#         self.corpus = [self.clean_text(x) for x in all_headlines]
#         # corpus : 말뭉치
#         # BOW : Bag of Words => 등장하는 모든 단어에 고유번호 지정
#         self.BOW ={} # 딕셔너리로 만듦
#         for line in self.corpus: #corpus에서 headline에서 추출한 문장들 불러오기
#             for word in line.split(): # 띄어쓰기를 기준으로 토큰화
#                 if word not in self.BOW.keys(): # 단어가 BOW에 있는지 확인 -> keys()는 굳이 안해도 됨
#                     self.BOW[word] = len(self.BOW.keys())
#         # 모델의 입력으로 사용할 데이터(self.data)
#         self.data = self.generate_sequence(self.corpus)
#         # sequence -> (1번,2번) => (3번) # 말이 이어지는 것 처럼 연결되도록

#     # 텍스트 시계열 -> BOW
#     # (단어1, 단어2) -> (단어3)                   
#     def generate_sequence(self, txt):
#         seq = [] # ([a,b],c) # a와 b를 넣으면 c가 나옴
#         for line in txt: # 전처리가 된 self.corpus 데이터 = 헤드라인
#             # 1단계
#             line = line.split() # 토큰화
#             # BOW(사전) -> key -> 고유번호 지정
#             # apple ->1, is-> 2, delicious -> 3
#             line_bow = [self.BOW[word] for word in line] # BOW[키값]으로 지정 => 1,2,3이 한 문장으로 문장을 고유번호화
#             # 단어 2개를 입력 -> 그 다음 단어가 정답(예측 대상)
#             # data =[]
#             # for i in range(len(line_bow)-2): # line_bow는 문장의 고유, -2인 이유는 문장 2개로 다음 1문장 예측 
#             #     # 0,1, 2 ~ [7,8]로 9를 예측, 8과 9는 예측 할 수 없음
#             #     # 0 -> 0, 1 => 2 / 1 -> 1, 2 => 3 / ... / n-3 -> n-2, n-1 => n-1
#             #     # d = ([a, b], c)
#             #     d = ([line_bow[i], line_bow[i+1]], line_bow[i+2]) # a, b를 사용해서, c를 예측 (순서대로)
#             #     data.append(d)
#             data = [([line_bow[i], line_bow[i+1]], line_bow[i+2])
#                     for i in range(len(line_bow)-2)]
#             # seq.extend(data)
#             seq += data
#         return seq
#     # 데이터 개수를 반환하는 함수
#     def __len__(self):
#           return len(self.data)

#     # 데이터를 불러오는 함수
#     def __getitem__(self, i):
#         data = np.array(self.data[i][0]) # (1번, 2번 단어)
#         label = np.array(self.data[i][0]) # (3번 단어)
#         return data, label

In [21]:
import torch.nn as nn
import torch

# 모델 정의

In [22]:
class LSTM(nn.Module):
    def __init__(self, num_embeddings) -> None:
        super().__init__()
        # 임베딩층 구성
        # [임베딩층]
        # I love pie
        # [0, 0, 0, 0, 0, 1...] # 희소표현
        # 자연어 처리 -> 단어 개수 -> 엄청 많다
        # 모델에 들어가는 대부분의 입력값이 0이 됨
        # => 희소표현 (학습 제대로 X) => 가중치 곱해도 0
        # 희소표현 => 밀집표현
        # 밀집표현을 위한 임베딩 층
        # num_embeddings => num_vocab => bow의 고유단어 갯수
        # data : ? , 2, 단어 갯수
        self.embed = nn.Embedding(
            num_embeddings=num_embeddings,
            embedding_dim=16,
        )
        # embedding : ?, 2, 16

        # LSTM 5개 층을 쌓음
        self.lstm = nn.LSTM(
            input_size=16,
            hidden_size=64,
            num_layers=5,
            batch_first=True,
        )
        # batch, sequence, feature
        #  (?) ,    2    ,   64
        # 분류 (fc)
        self.fc1 = nn.Linear(128, num_embeddings) # => 다시 고유단어 갯수로 분류
        self.fc2 = nn.Linear(num_embeddings, num_embeddings)
        # 정확도를 높이기 위한 분류기 한 더 사용

        # 활성화 함수
        self.relu = nn.ReLU()
    
    # 순전파
    def forward(self, x):
        # bow 고유단어 갯수 (2, ...) -> 16 (2, 16)
        x = self.embed(x) # 희소 -> 밀집
        # LSTM 모델의 예측값
        x, _ = self.lstm(x) # 전체 출력 상태 + 셀 상태값
        # (2, 16) -> (2, 64)
        x = torch.reshape(x, (x.shape[0], -1)) # batch, 2 * 64 => batch, 128
        # 분류기
        x = self.fc1(x) # 128 -> 단어 수
        x = self.relu(x)
        x = self.fc2(x) # return
        return x

In [23]:
# class LSTM(nn.Module):
#     def __init__(self, num_embeddings) -> None:
#         super().__init__()
#         # 임베딩층 구성
#         # [임베딩층]
#         # I love pie
#         # [0,0,0,0,0,1....] # 희소표현
#         # 자연어 처리는 단어의 개수가 많은데 0,1로 단어가 있는지 판단하기에는 너무 많다
#         # 모델에 들어가는 대부분의 입력값이 0이 됨
#         # => 희소표현 (학습이 제대로X) => 가중치 곱해도 0
#         # 희소표현 => 밀집 표현
#         # 밀집을 위한 임베딩 층
#         # num_embeddings => num_vocab => bow의 고유 단어 개수
#         # data: 7, 2, 단어 개수
#         self.embed = nn.Embedding(
#             num_embeddings=num_embeddings,
#             embedding_dim=16,
#         )
#         # embedding: 7, 2, 16
#         # LSTM 5개 층을 쌓음
#         self.lstm = nn.LSTM(
#             input_size=16,
#             hidden_size=64,
#             num_layers=5,
#             batch_first=True,
#         )
#         # batch, sequence, feature
#         #  (?),     2    ,   64 
#         # 분류(fc)
#         self.fc1 = nn.Linear(128, num_embeddings) # => 다시 고유단어 갯수로 분류
#         self.fc2 = nn.Linear(num_embeddings, num_embeddings)
#         # 정확도를 높이기 위한 분류기 한 더 사용

#         # 활성화 함수

#         self.relu = nn.ReLU()

#     # 순전파
#     def forward(self, x):
#         # bow 고유단어 개수(2, ...) -> 16 (2, 16)
#         x = self.embed(x) # 희소 -> 밀집
#         # LSTM 모델의 예측값
#         x, _ = self.lstm(x) # 전체 출력 상태 + 셀 상태값
#         # (2, 16) -> (2,64)
#         x = torch.reshape(x, (x.shape[0], -1)) # batch, 2 * 64 => batch, 128
#         # 분류기

#         x = self.fc1(x) # 128 -> 단어 수
#         x = self.relu(x)
#         x = self.fc2(x) # return
#         return x

# 학습

In [24]:
from tqdm.notebook import tqdm # 진척도

from torch.utils.data import DataLoader # 배치사이즈로 끊어서 데이터 제공
from torch.optim.adam import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [25]:
dataset = TextGeneration()

In [26]:
loader = DataLoader(dataset, batch_size=64)

In [27]:
model = LSTM(num_embeddings=len(dataset.BOW)).to(device)
optim = Adam(model.parameters(), lr=0.001) # 0.001 ~ 0.00001

In [28]:
# for epoch in range(200):
for epoch in range(50):
    iterator = tqdm(loader)
    # data: 입력값 (1번째, 2번째 단어) / label : 정답값 (3번째 단어)
    for data, label in iterator:
        optim.zero_grad() # 기울기 초기화
        # 모델의 예측값
        # pred = model(torch.tensor(data, dtype=torch.long).to(device))
        data = torch.tensor(data, dtype=torch.long).to(device)
        pred = model(data)
        # 정답 레이블
        label = torch.tensor(label, dtype=torch.long).to(device)
        # 오차
        loss = nn.CrossEntropyLoss()(pred, label)
        # 오차 역전파
        loss.backward()
        optim.step()

        iterator.set_description(f"epoch{epoch} / loss: {loss.item()}")

torch.save(model.state_dict(), "lstm.pt")

  0%|          | 0/677 [00:00<?, ?it/s]

<ipython-input-28-4f76160751c0>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.long).to(device)
<ipython-input-28-4f76160751c0>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.long).to(device)


  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

...

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

In [29]:
# 문장 만들기 -> 2개 이상의 단어로 구성된 문장을 넣으면 => 다음 단어를 유추
def generate(model, BOW, string="finding an ", strlen=10):
    print(f"input word : {string}")

    with torch.no_grad(): # 기울기 계산 끄고,
        for p in range(strlen):
            # 입력한 문장을 텐서화
            # [BOW[w] for w in string.split()] -> string을 스페이스 단위로 분할, 해당 토큰(단어)들이 BOW에서 어디에 있는지 확인
            # torch.tensor(..., dtype=torch.long).to(device) -> 모델이 이해할 수 있는 텐서로 변환
            words = torch.tensor([BOW[w] for w in string.split()], dtype=torch.long).to(device)
            # 모델의 입력으로 입력으로 사용하기 위한 배치 차원을 추가
            input_tensor = torch.unsqueeze(words[-2:], dim=0) # 마지막 두 단어만 추출해서 -> 두 단어를 배치 차원 반영하여 분석
            output = model(input_tensor) # 모델을 통해 예측
            # torch.argmax(output) -> 가장 확률값이 높은 위치의 텐서의 인덱스
            # 0~전체사전의 길이만큼의 텐서. 확률값 -> 단어의 고유번호
            # (...).cpu().numpy() -> (cuda-gpu) => cpu => numpy 배열
            output_word = (torch.argmax(output).cpu().numpy())
            string += list(BOW.keys())[output_word]
            string += " "
        
    print(f"예측 문장 : {string}")

In [30]:
model.load_state_dict(torch.load("lstm.pt", map_location=device))
pred = generate(model, dataset.BOW)

input word : finding an 
예측 문장 : finding an young bean young think a doughnut in living and the 


In [31]:
pred = generate(model, dataset.BOW, 'red apple ')

input word : red apple 
예측 문장 : red apple and the young south stating sugarfish and the young south 


In [32]:
pred = generate(model, dataset.BOW, 'blue sky ')

input word : blue sky 
예측 문장 : blue sky a pill in the picture think rise and tragedy in 


In [33]:
pred = generate(model, dataset.BOW, 'smart computer ')

input word : smart computer 
예측 문장 : smart computer and the young south stating sugarfish and the young south 


In [34]:
pred = generate(model, dataset.BOW, 'hello world ')

input word : hello world 
예측 문장 : hello world maine the doughnut in living and the young south stating 
